# Bedrock での 質問と回答

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

質問応答（QA）は、自然言語で表現された事実に関連する質問に対する回答を抽出する重要なタスクです。通常、QA システムは、構造化データまたは非構造化データを含むナレッジベースに対するクエリを処理し、正確な情報を含む回答を生成します。特にエンタープライズのユースケースでは、有用で信頼性が高く、信頼できる質問応答システムを開発するには、高い精度を確保することが重要です。

Titan や Claude のような生成系 AI モデルは、確率分布を使用して質問に対する回答を生成します。 これらのモデルは膨大な量のテキストデータに基づいてトレーニングされているため、シーケンスの次に何が起こるか、特定の単語の後にどの単語が続くかを予測できます。ただし、データには常にある程度の不確実性があるため、これらのモデルではすべての質問に対して正確または決定論的な回答を提供することはできません。企業は、ドメイン固有の独自データを照会し、その情報を使用して質問に答える必要があり、モデルのトレーニングを受けていないデータにも対応する必要があります。 

このモジュールでは、Bedrock Titan モデルを使用して、クエリに対する応答を提供する方法を示します。

今回の例では、コンテキストなしでモデルを実行し、手動でコンテキストを指定してみます。こでは `RAG` 拡張は行われていません。この手法は、短いドキュメントやシングルトンのアプリケーションでは機能しますが、モデルに送信されるプロンプトにすべて収まらない大規模なエンタープライズドキュメントが存在する企業レベルの質問回答には対応できない場合があります。 


### チャレンジ
- モデルに事実に基づいて質問の回答を返させる方法

### 提案
上記の課題に対して、このノートブックでは以下の戦略を提案しています。
#### ドキュメントの準備
質問に答えるには、ドキュメントを処理してドキュメントストアインデックスに保存する必要があります。
- ここでは、モデルに関連するすべてのコンテキストを含むリクエストを送信し、応答を待ちます。


## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みと ⚠️ **whether you might need to make any changes** についての詳細は、[Bedrock boto3 setup ノートブック](../00_Intro/bedrock_boto3_setup.ja.ipynb)  を参照してください。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


#----⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## Section 1: モデルの知識を用いた質問応答
このセクションでは、Bedrock サービスが提供するモデルを使用し、トレーニングフェーズで得た知識に基づいて質問に回答するようにします。

この Notebook では、Amazon Bedrock クライアントの `invoke_model() `メソッドを使用します。このメソッドを使用するための必須パラメータは、Amazon Bedrock モデル ARN を表す `modelId` と、タスクのプロンプトである `body` です。`body` プロンプトは、選択した基盤モデルプロバイダーによって異なります。これについては、以下で詳しく説明します。

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```


## シナリオ

我々はモデルにタイヤ交換の情報を提供してもらっているという状況をモデル化しようとしています。まず、トレーニングデータに基づいてモデルに問い合わせ、特定の車種とモデルに関する回答を求めます。このテクニックは 'Zero Shot` と呼ばれています。そのモデルが我々の特定の車に関連している回答を返しているように見えても、実際にはハルシネーションを起こしていることにすぐに気付くでしょう。それがわかるのは、偽の車を問い合わせた場合にも、ほとんど同じようなシナリオと回答が返ってくるからです。

このような状況では、特定の車種とモデルに関する追加データでモデルのトレーニングを補強する必要があります。このノートブックでは、データを補強するために外部ソースは使用せず、RAG ベースの補強システムがどのように機能するかをシミュレートします。

最終テストを実施するために、私たちはマニュアルの詳細を提供して、特定のクルマについてタイヤ交換がどのように機能するかを説明し、モデルからのレスポンスをテストします。

## タスク

プロセスを開始するには、Bedrock が提供するモデルのいずれかを選択します。今回のユースケースでは、Titan を選択します。これらのモデルは、車に関する一般的な質問に答えることができます。

例えば、Audi のパンクしたタイヤの交換方法を Titan モデルに尋ねます。

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way.'

Question: How can I fix a flat tire on my Audi A8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

#### レスポンスを生成するために、JSON bodyを渡してモデルを呼び出してみましょう。

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # モデルプロバイダーから違うモデルを使用する際は、ここを変更してください。
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

このモデルは、パンクしたタイヤを交換するプロセスの概要を説明する回答を提供することができますが、同じ説明はどの車にも有効です。残念ながら、スペアタイヤのない Audi A8 にとっては正しい答えではありません。これは、モデルが車のタイヤ交換に関する説明を含むデータで学習されているためです。

この問題の別の例は、全く偽の車のブランドとモデル、例えば Amazon Tirana についての質問をすることで確認することができます。

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # モデルプロバイダーから違うモデルを使用する際は、ここを変更してください。
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

ご覧の通り、モデルが出した答えはもっともらしいですが、それは自転車に対するものであり、自動車に対するものではありません。モデルは Amazon Tirana を自転車と仮定したのです。モデルはハルシネーションを起こしています。

どうすればこの問題を解決し、モデルが私の車種に有効な固有の指示に基づいて答えを出すようにできるのでしょうか？

2020年に Facebook が行った研究では、プロンプトの一部として追加の知識ベースを提供することで、LLM の知識をその場で補強できることがわかりました。このアプローチはRAG（Retrieval Augmented Generation）と呼ばれています。

我々のアプリケーションを改善するために、これをどのように使えるか見てみましょう。

以下は、Audi A8 のマニュアルの抜粋です。（実際は本当のマニュアルではありませんが、そう仮定します）この文書は Titan Large のプロンプトにすっぽり収まるくらい短いため便利です。


```
Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```


次に、このテキストを元の質問とともにプロンプトに「埋め込み」ます。またプロンプトは、コンテキストとして提供された情報のみを見るよう、モデルにヒントを与えるように作られています。

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Audi or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

#### 抜粋全体を、質問と一緒にモデルに渡してみましょう。

In [ ]:
question = "How can I fix a flat tire on my Audi A8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

##### boto3 経由でモデルを起動し、レスポンスを生成します。

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

モデルがコンテキストを理解し、あなたに関連する答えを生成するのに時間がかかるため、何秒間かレスポンスを待たなければなりません。これは良くないユーザー体験につながる可能性があります。

Bedrockは、モデルがトークンを生成している間にサービスが出力を生成するようなストリーミング機能もサポートしています。以下はその例です。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

## まとめ

レスポンスはタイヤを交換する方法を要約し、ステップバイステップでタイヤを交換する方法を指示していることがわかります。この単純な例は、`RAG` や拡張プロセスを活用して、どのように調整された回答を返すことができるかを示しました。